In [6]:
import numpy as np
import pandas as pd
import statsrat as sr
from statsrat import latent_cause as lc
from statsrat.expr.predef.pvl_no_iti import spont_rec_imp as task
from plotnine import *

In [7]:
#model = lc.model(name = 'constant', kernel = lc.kernel.constant)
model = lc.model(name = 'power', kernel = lc.kernel.power)
print(model.pars)
perform_oat = False
save_plots = True

method = 'local_vb'
#method = 'particle'

prior_tau2_x = 1.0
prior_nu_x = 5.0
prior_tau2_y = 0.2
prior_nu_y = 1.0
alpha = 1.0
stick = 0.0
power = 0.3

par_vals = {'constant': [prior_tau2_x, prior_nu_x, prior_tau2_y, prior_nu_y, stick, alpha, 5.0],
            'power': [power, prior_tau2_x, prior_nu_x, prior_tau2_y, prior_nu_y, stick, alpha, 5.0]}

               min   max  default  \
power         0.00   5.0      1.0   
prior_tau2_x  0.01  10.0      1.0   
prior_nu_x    1.00  10.0      5.0   
prior_tau2_y  0.01  10.0      1.0   
prior_nu_y    1.00  10.0      5.0   
stick         0.00   5.0      1.0   
alpha         0.00  15.0      1.0   
resp_scale    0.00  10.0      1.0   

                                                    description  
power         decay rate for power law SCRP; higher -> favor...  
prior_tau2_x                 prior hyperparameter for eta for x  
prior_nu_x                   prior hyperparameter for eta for x  
prior_tau2_y                 prior hyperparameter for eta for y  
prior_nu_y                   prior hyperparameter for eta for y  
stick                                  stickiness for CRP prior  
alpha         concentration parameter; higher -> tend to inf...  
resp_scale           scales softmax/logistic response functions  


In [ ]:
print('spontaneous recovery \n')

trials_exp = task.make_trials('experimental')
trials_ctl = task.make_trials('control')

if method == 'particle':
    ds_exp = model.particle_filter(trials_exp, par_vals[model.name])
    ds_ctl = model.particle_filter(trials_ctl, par_vals[model.name])
    #sr.multi_plot([ds_exp, ds_ctl], var = 'b', dodge_width = 0.2).draw()
    #sr.multi_plot([ds_exp, ds_ctl], var = 'mean_N', dodge_width = 0.2).draw()
else:
    ds_exp = model.simulate(trials_exp, par_vals[model.name])
    ds_ctl = model.simulate(trials_ctl, par_vals[model.name])
    #sr.multi_plot([ds_exp, ds_ctl], var = 'b', dodge_width = 0.2).draw()
    #sr.learn_plot(ds_exp, var = 'E_log_prior', sel = {'z_name': ['0', '1', '2']}).draw()
    #sr.multi_plot([ds_exp, ds_ctl], var = 'N', dodge_width = 0.2).draw()

plot_b = sr.multi_plot([ds_exp, ds_ctl],
                        var = 'b',
                        draw_points = True,
                        dodge_width = 0.2,
                        rename_coords = {'schedule': 'group'},
                        rename_schedules = {'control': 'control', 'experimental': 'delay'},
                        sel = 2*[{'t': ds_exp['stage_name'].isin(['cond', 'extinction', 'test'])}])
plot_b += scale_color_manual(values = {'delay': 'red', 'control': 'blue'})
plot_b.draw()
if save_plots:
    plot_b.save(filename = 'plots/latent_cause_spont_rec_b_' + model.name + '.jpeg', dpi = 300)
    
oat_name = list(task.oats.keys())[0]
print(task.oats[oat_name].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(task.oats[oat_name].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

spontaneous recovery 



In [ ]:
if method == 'local_vb':
    print('experimental condition \n')

    df_exp = ds_exp['z'].to_dataframe()
    table_exp = pd.crosstab(df_exp['z'], df_exp['ex'])
    print(ds_exp['z'].values)
    print(table_exp)
    
    print('control condition \n')

    df_ctl = ds_ctl['z'].to_dataframe()
    table_ctl = pd.crosstab(df_ctl['z'], df_ctl['ex'])
    print(ds_ctl['z'].values)
    print(table_ctl)
    
    plot_r = sr.learn_plot(ds_exp,
                           var = 'phi_x',
                           rename_coords = {'z_name': 'latent_cause'},
                           sel = {'z_name': ['0', '1', '2']})
    plot_r += ylab('r')
    plot_r.draw()
    if save_plots:
        plot_r.save(filename = 'plots/latent_cause_spont_rec_r_' + model.name + '.jpeg', dpi = 300)

In [ ]:
if perform_oat:
    n_dict = {'local_vb': 1, 'particle': 10}
    oat_result = sr.perform_oat(model = model,
                                experiment = task,
                                n = n_dict[method],
                                minimize = False,
                                sim_type = method,
                                max_time = 60)
    print(np.round(oat_result[0], 5))
    print(np.round(oat_result[1], 3))